In [ ]:
import sympy as sm
import pybolano as bl
from sympy.physics.quantum import Dagger
from sympy.physics.quantum.boson import BosonOp
from sympy.physics.quantum.operatorordering import normal_ordered_form
import matplotlib.pyplot as plt
import matplotlib.gridspec as mplgs
import matplotlib.colors as mplcl
import time
from tqdm import tqdm
import numpy as np
import random
bl.mp_config["num_cpus"] = 8

b, bd = bl.ops()

b_s = BosonOp(r"b_\mathrm{S}")
bd_s = Dagger(b_s)

**Benchmark a:** normal ordering of $N$ monomials containing $n_\mathrm{ladder}$ ladder operators and a set $k$ of subscripts.

In [ ]:
def get_data(n_ladder, k, N):
    t_bl = []
    t_s = []
    monomials = []
    for _ in range(1, N+1):
        
        q = bl.random_ladder(n_ladder, k)
        q_s = bl.to_sympy_physics(q)
        monomials.append(q)
        
        t_i = time.time()
        bl.normal_ordering(q)
        dur = time.time()-t_i
        t_bl.append(dur)

        t_i = time.time()
        normal_ordered_form(q_s, recursive_limit=100000) 
        dur = time.time()-t_i
        t_s.append(dur)
        
    return monomials, t_bl, t_s

n_ladder = 10
k = [1,2]
N = 1000

# monomials, t_bl, t_s = get_data(n_ladder=n_ladder, k=k, N=N)
# np.savetxt("a-t_bl", t_bl)
# np.savetxt("a-t_s", t_s)

In [ ]:
plt.rcParams["font.size"] = 20
plt.rcParams["axes.linewidth"] = 2
fig = plt.figure(figsize=(6, 4), constrained_layout=True)
gs = mplgs.GridSpec(1, 1, fig)
axes = [fig.add_subplot(gs[i]) for i in range(1)]

bg_color = ("yellow",0.2)

ax = axes[0]
t_bl = np.loadtxt("a-t_bl")
t_s = np.loadtxt("a-t_s")
N_lst = range(len(t_bl))
ax.scatter(N_lst, t_bl, c=bg_color, marker="s", edgecolor="b", s=12, label="pyBoLaNO")
ax.scatter(N_lst, t_s, c="r", marker="o", s=5, label="SymPy")

ax.set_xlabel("test #")
ax.set_ylabel("execution time (s)")
ax.set_facecolor(bg_color)

ax.set_yscale("log")

plt.tick_params(width = 2, direction = "in")
fig.legend(loc = "upper center", bbox_to_anchor = (0.5, 0.05), ncol=2, framealpha=0,
           fontsize = 18)

Every case where `SymPy` wins is where the input is already normal-ordered to begin with. `pyBoLaNO` takes a bit longer since it does not check whether the input is normal-ordered. `SymPy`'s implementation does this as a consequence of its algorithm.

In [ ]:
# for i in np.where(t_s<t_bl)[0]:
#     print(sm.latex(monomials[i]))


---

**Benchmark b:** Difference between the averages of **Benchmark a** results for varying $n_\mathrm{ladder}$ and $|k|$.

In [ ]:
n_ladder_lst = list(range(1, 21))
k_lst = []
for i in range(1, 21):
    k = []
    for j in range(1, i+1):
        k.append(j)
    k_lst.append(k)
    
np.savetxt("b/n_ladder", n_ladder_lst)
np.savetxt("b/k_size", [len(k) for k in k_lst])

res = []
for iter, n_ladder in enumerate(n_ladder_lst):
    ress = []
    for iiter, k in enumerate(k_lst):
        print(f"n ({iter+1}/{len(n_ladder_lst)}), k ({iiter+1}/{len(k_lst)})")
        _, t_bl, t_s = get_data(n_ladder=n_ladder, k=k, N=N)
        mean_t_bl = np.mean(t_bl)
        mean_t_s = np.mean(t_s)
        ress.append(mean_t_s - mean_t_bl)
    res.append(ress)
    print("---")
    
np.savetxt("b/res", res)

In [ ]:
x = np.loadtxt("b/n_ladder")
y = np.loadtxt("b/k_size")
x, y = np.meshgrid(x, y)
z = np.loadtxt("b/res")

plt.imshow(z)